# OpenStreetMap Data Wrangling

<li>Name: Jubin Soni</li>
<li>Udacity DAND Nanodegree Project 3</li>

### Map Area
New York, NY, United States
<ul>
<li>Data Source: https://www.openstreetmap.org/relation/175905</li>
<li>This map is where I say so I am more interested in what this study reveals by querying the database. I would also like to contribute to its improvement on OpenStreetMap.org.</li>
<li>The downloaded OSM file is 2.7GB in size and it is difficult to process so much data so first I have created a sample of 270MB from this file and used it for initial analysis.</li>
</ul>

### Creating a sample of original OSM file

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Following code shortens the OSM_FILE and stores it in SAMEPLE_FILE by dividing factor of 'k'
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "new-york.osm"  # Replace this with your osm file
SAMPLE_FILE = "new-york-sample.osm"

k = 100 # Parameter: take every k-th top level element

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

### Problems Encountered in the Map

After downloading and creating sample from the NYC OSM data. I noticed few problems with the data.
<ul>
<li> Abbreviated street names ("E 14th St", "Myrtle Ave", "Father Capodanno Blvd", "White Plains Rd") </li>
<li> Inconsistent zip codes ("10307:10312", "11220; 11204") </li>
<li> Overabbreviated and Inconsistent name_types ("Blvd;Blvd;Pl;Blvd") </li>
<li> Inorrect zip codes ("07544", "06853"). All zip codes in NYC start from 10 or 11 </li>
<li> Street names in second level “k” tags pulled from Tiger GPS data and divided into segment</li>
</ul>

### Correcting these problems (Audit.py)

In [3]:
def update(name, mapping): 
    words = name.split()
    for w in range(len(words)):
        if words[w] in mapping:
            if words[w].lower() not in ['suite', 'ste.', 'ste']: 
                # For example, don't update 'Suite E' to 'Suite East'
                words[w] = mapping[words[w]]
                name = " ".join(words)
    return name

def correct_abbreviations(problem_field, tree, mapping):
    for tag in tree.iter('tag'):
        if problem_field in tag.attrib['k']:
            words = tag.attrib['v']
            updated_words = ""
            for word in words.split(" "):
                if (word in mapping):
                    word = mapping[word]
                updated_words = "".join(word)
            tag.attrib['v'] = updated_words



import xml.etree.cElementTree as ET
SAMPLE_FILE = "new-york-sample.osm"
tree = ET.parse(SAMPLE_FILE)

street_mapping = {"St":"Street", "St.":"Street", "Ave": "Avenue", "ave": "Avenue", "Blvd": "Boulevard", "Rd": "Road"}
name_type_mapping = {"Pkwy": "Parkway", "St": "Street", "St.":"Street", "Ave": "Avenue", "ave": "Avenue", "Blvd": "Boulevard", "Rd": "Road"}

correct_abbreviations('street', tree, street_mapping)
correct_abbreviations('name_type', tree, name_type_mapping)
tree.write(SAMPLE_FILE)

### Data Cleaning (Data.py): Processing the OSM file and storing in separate CSVs

In [10]:
#data.py
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus
import schema_

OSM_PATH = SAMPLE_FILE #SAMPLE_FILE = "new-york-sample.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema_.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        for field in NODE_FIELDS:
            node_attribs[field] = element.attrib[field]
        
        for tag in element.iter('tag'):
            tag_dict = {}
            tag_dict['id'] = element.attrib['id']
            if PROBLEMCHARS.match(tag.attrib['k']):
                pass
            elif ':' in tag.attrib['k']:
                tag_splittable = tag.attrib['k']
                tag_dict['type'] = tag_splittable.split(':')[0]
                tag_dict['key'] = tag_splittable.split(':',1)[1]
            else:
                tag_dict['type'] = 'regular'
                tag_dict['key'] = tag.attrib['k']
                
            tag_dict['value'] = tag.attrib['v']
            tags.append(tag_dict)
        
        return {'node': node_attribs, 'node_tags': tags}
            
    if element.tag == 'way':
        for field in WAY_FIELDS:
            way_attribs[field] = element.attrib[field]
        
        position_counter = 0
        for nd in element.iter('nd'):
            nd_dict = {}
            nd_dict['id'] = element.attrib['id']
            nd_dict['node_id'] = nd.attrib['ref']
            nd_dict['position'] = position_counter
            position_counter += 1
            way_nodes.append(nd_dict)
            
        for tag in element.iter('tag'):
            tag_dict = {}
            tag_dict['id'] = element.attrib['id']
            if PROBLEMCHARS.match(tag.attrib['k']):
                pass
            elif ':' in tag.attrib['k']:
                tag_dict['type'] = tag.attrib['k'].split(':')[0]
                tag_dict['key'] = tag.attrib["k"].split(':',1)[1]
            else:
                tag_dict['type'] = 'regular'
                tag_dict['key'] = tag.attrib['k']
            #value
            tag_dict['value'] = tag.attrib['v']
            tags.append(tag_dict)
            
    return {'way': way_attribs, 'way_tags': tags, 'way_nodes': way_nodes}



# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, SCHEMA) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)
    print ("Processed OSM file and 5 new files have been created:\n- nodes.csv\n- nodes_tags.csv\n- ways.csv\n- ways_tags.csv\n- ways_nodes.csv")


Processed OSM file and 5 new files have been created:
- nodes.csv
- nodes_tags.csv
- ways.csv
- ways_tags.csv
- ways_nodes.csv


## SQLite3 (Data Exploration and Gathering insights)

The five files generated from above script were fed into SQLite3 and used to answer few questions about NYC Metro Area.

### Number of nodes

```SQL
sqlite> select count(*) from nodes;

Output: 230102
```

### Number of ways

```SQL
sqlite> select count(*) from ways;

Output: 36066
```

### Number of unique users

```SQL
sqlite> select count(distinct(e.uid)) from
(select uid from nodes union all select uid from ways) e;

Output: 1252
```

### Top 10 contributing users

```SQL
sqlite> select e.user, count(*) as num from
(select user from nodes union all select user from ways) e
group by e.user
order by num desc
limit 10;

Output:

|e.user               |num  |
|---------------------|-----|
|                     |48886|
|Rub21_nycbuildings   |9347 |
|ingalls_nycbuildings |6267 |
|MySuffolkNY          |6186 |
|woodpeck_fixbot      |5806 |
|minewman             |4912 |
|Northfork            |4140 |
|ediyes_nycbuildings  |2711 |
|lxbarth_nycbuildings |2350 |
|---------------------|-----|
```

### Number of users apprearing only once (having 1 post)

```SQL
sqlite> select count(*) from
(select e.user, count(*) as num from
(select user from nodes union all select user from ways) e
group by e.user
having num = 1);

Output: 56
```

## Additional Data Exploration

### Top 10 appearing amenities

```SQL
sqlite> select value, count(*) as num
from nodes_tags
where key = 'amenity'
goup by value
order by num desc
limit 10;

Output:

|value            |num  |
|-----------------|-----|
|bicycle parking  |51   |
|restaurant       |33   |
|school           |31   |
|place_of_worship |13   |
|fast_food        |15   |
|cafe             |13   |
|bar              |8    |
|pharmacy         |6    |
|bicycle_rental   |5    |
|fire_station     |5    |
|-----------------|-----|
```

### Top 10 cuisines

```SQL
sqlite> select nodes_tags.value, count(*) as num
from nodes_tags join
(select distinct(id) from nodes_tags where value = 'restaurant') r
on nodes_tags.id = r.id
where nodes_tags.key = 'cuisine'
group by nodes_tags.value
order by num desc;

Output:

|nodes_tags.value         |num  |
|-------------------------|-----|
|american                 |2    |
|Mexican                  |2    |
|american;regional;burger |1    |
|asian                    |1    |
|chinese                  |1    |
|diner                    |1    |
|french                   |1    |
|frozen yogurt            |1    |
|ice_cream                |1    |
|japanese                 |1    |
|korean                   |1    |
|mexican                  |1    |
|pizza                    |1    |
|-------------------------|-----|
```

### Future prospects and Analysis

After this review of the data it is obvious that the New York City area is very complete, and although I believe it has been well cleaned for the purposes of this exercise I think further cleaning the data will reveal more interesting insights. For example in the 'Top 10 Cuisines' one of the nodes_tags.value has multiple comma-separated value and in 'Top 10 contributors' the first field is blank. This shows that in the entire file there can be more such values and these minor corrections can be done. Also some more complicated analysis can be done using SQL like most occuring (worshipped) religion in OSM, and most active user.

# Conclusion

It interests me to notice a fair amount of GPS data makes it into OpenStreetMap.org on account of users’ efforts, whether by scripting a map editing bot or otherwise. With a rough GPS data processor in place and working together with a more robust data processor similar to data.py and audit.py. I think it would be possible to input a great amount of cleaned, validated and corrected data to OpenStreetMap.org.